In [1]:
## specify train data
train_data = ""
## specify validation data
dataset = ""
# specify column for ratings (non-negative intefer scale)
score = ""
# specify column for original scores
score_og = ""
# specify for text
text = ""

## model_no (cf. training.py)
model_no = 0
# model from which epoch?
epoch = 1

In [2]:
### importing libraries
import torch  # ML lib. allows computation of tensors on GPUs
import os
import json
import time  # to time execution time
import pandas as pd  # data analysis and manipulation
import numpy as np  # for mathematical operations
import scikitplot as skplt  # import scikitplot as skplt
import matplotlib.pyplot as plt  # plotting

plt.style.use("ggplot")
from transformers import BertTokenizer  # tokenizes sequences of strings
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import SequentialSampler

# create tensors from data and make them accesible for model
from transformers import BertForSequenceClassification  # our model
from sklearn.metrics import f1_score  # for model evaluation

# helper function
# flat_values function
def flat_values(preds, labels):
  '''
  Creates an array of maxima for each element in a list of arrays.
  Creates array from list.
  :param preds: list of arrays of numeric values (e.g. prediction scores)
  :param labels: list of arrays (potentially with only one element) of numerric values
  :return pred_flat: array of the maxima of the respective arrays in preds
  :return labels_flat: flattened list (e..g transformed to np.array while preserving every element of
                       the original arrays of the list)
  '''
  # list of keys of the highest scores, i.e. predictions
  pred_flat = np.argmax(preds, axis=1).flatten()
  # list of true labels
  labels_flat = labels.flatten()
  return pred_flat, labels_flat

OSError: [WinError 127] Die angegebene Prozedur wurde nicht gefunden. Error loading "C:\Users\NALMPI\anaconda3\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [3]:
# read data
df = pd.read_csv("./data/processed/{}.csv".format(dataset))

# sample size
n_val = len(df)

# load model specs
with open(
    "./data/temp/training/{}/model_specs_{}.json".format(dataset, model_no)
) as f:
    model_parameters = json.load(f)

NameError: name 'pd' is not defined

In [4]:
model_type = model_specs["model_type"]
MAX_LEN = model_specs["MAX_LEN"]
BATCH_SIZE = model_specs["BATCH_SIZE"]
LR = model_specs["LR"]
EPS = model_specs["EPS"]
device_name = model_specs["device_name"]

# create folder to store results from this file
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Creating directory: " + directory)
        
        
        

NameError: name 'model_specs' is not defined

In [5]:
createFolder("./results/sentiment_analysis/evaluation/")
createFolder("./results/sentiment_analysis/evaluation/{}/".format(dataset))

# specify device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NameError: name 'createFolder' is not defined

In [6]:
# define tokenizers
tokenizer = BertTokenizer.from_pretrained(model_type, do_lower_case=True)

## encode text data
encoded_data_val = tokenizer.batch_encode_plus(
    df[text].values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=MAX_LEN,
    truncation=True,
    return_tensors="pt",
)

## prepare input for model
input_ids_val = encoded_data_val["input_ids"]
attention_masks_val = encoded_data_val["attention_mask"]
labels_val = torch.tensor(df[score].values, dtype=torch.long)

NameError: name 'BertTokenizer' is not defined

In [7]:
# TensorDataset combines prepared model inputs in a tensor for the model
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

dataloader_val = DataLoader(
    dataset_val, sampler=SequentialSampler(dataset_val), batch_size=BATCH_SIZE
)

## load the number of labels required for initial model
NUM_LABELS = np.load(
    "./data/temp/training/{}/num_labels.npy".format(train_data)
)

NameError: name 'TensorDataset' is not defined

In [8]:
model = BertForSequenceClassification.from_pretrained(
    model_type,
    num_labels=NUM_LABELS,
    output_attentions=False,
    output_hidden_states=True,
)

## overwrite model parameters with finetuned parameters
model.load_state_dict(
    torch.load(
        f"./results/sentiment_analysis/training/{train_data}/BERT_{model_no}_epoch_{epoch}.model"
    )
)
model.to(device)

## set model to evaluation mode
model.eval()

NameError: name 'BertForSequenceClassification' is not defined

In [9]:
# Evaluation Function

def evaluate(dataloader_val):
    # track batch loss
    val_loss_set = []
    # initiate cumulative loss
    loss_val_total = 0
    # initiate arrays for predicted and true labels
    predictions, true_vals = [], []
    # evlaution loop
    for batch in dataloader_val:
        # add batch to GPU
        batch = tuple(b.to(device) for b in batch)
        # Unpack the inputs from  dataloader
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "labels": batch[2],
        }
        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # forward pass: calculate logit predictions
            outputs = model(**inputs)
        # the output consists of the batch loss (index [0])
        # and the predicted scores for all labels (index [1])
        loss = outputs[0]
        val_loss_set.append(loss.item())
        logits = outputs[1]
        # cumulative loss over all batches
        loss_val_total += loss.item()
        # send model outputs to cpu and transform to np.array
        logits = logits.detach().cpu().numpy()
        label_ids = inputs["labels"].cpu().numpy()
        # store predictions and true labels
        predictions.append(logits)
        true_vals.append(label_ids)
    # divide cumulative batch loss by number of batches
    loss_val_avg = loss_val_total / len(dataloader_val)
    # predictions and true_vals are lists of arrays
    # concatenate to create arrays of values
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals, val_loss_set

In [10]:
# evaluate model in validation dataset
start = time.time()
val_loss, predictions, true_vals, val_loss_set = evaluate(dataloader_val)
end = time.time()
wall_time = end - start

# compute and print validation loss and f1 score
predictions_flat, labels_flat = flat_values(predictions, true_vals)
val_f1 = f1_score(predictions_flat, labels_flat, average="weighted")
print("Validation loss: {}".format(val_loss))
print("F1 Score (Weighted): {}".format(val_f1))

NameError: name 'time' is not defined

In [11]:
predictions = predictions_flat

NameError: name 'predictions_flat' is not defined

In [12]:
# confusion matrix of validation data after final epoch
skplt.metrics.plot_confusion_matrix(
    df[score_og], predictions, title="Confusion Matrix"
)
plt.savefig(
    "./results/sentiment_analysis/evaluation/{}/conf_model_{}_epoch_{}.png".format(
        dataset, model_no, epoch
    )
)


NameError: name 'skplt' is not defined

In [ ]:
# accuracy
accuracy = np.sum(predictions_flat == labels_flat) / len(labels_flat)

# load or create a table where results for different models are stored
# shall be used with one test set and differet models (model numbers)
# helpful for hyperparameter tuning
try:
    evaluation_table = pd.read_csv(
        "./results/sentiment_analysis/evaluation/evaluation_table.csv",
        index=False,
    )
except:
    col_names = [
        "Model",
        "Batch Size",
        "Max Length",
        "Optimizer",
        "Learning Rate",
        "Tolerance",
        "n",
        "validation loss (epoch 1)",
        "validation loss (epoch 2)",
        "validation loss (epoch 3)",
        "validation loss (epoch 4)",
        "f1 score (epoch 1)",
        "f1 score (epoch 2)",
        "f1 score (epoch 3)",
        "f1 score (epoch 4)",
        "data",
        "training data",
        "gpu",
    ]
    evaluation_table = pd.DataFrame(columns=col_names)

# insert values into the respective row of info table
if epoch == 1:
    evaluation_table.loc[model_no] = [
        model_type,
        BATCH_SIZE,
        MAX_LEN,
        "AdamW",
        LR,
        EPS,
        n_val,
        val_loss,
        "n/a",
        "n/a",
        "n/a",
        val_f1,
        "n/a",
        "n/a",
        "n/a",
        dataset,
        train_data,
        device_name,
    ]
elif epoch == 2:
    evaluation_table.loc[model_no] = [
        model_type,
        BATCH_SIZE,
        MAX_LEN,
        "AdamW",
        LR,
        EPS,
        n_val,
        "n/a",
        val_loss,
        "n/a",
        "n/a",
        "n/a",
        val_f1,
        "n/a",
        "n/a",
        dataset,
        train_data,
        device_name,
    ]
elif epoch == 3:
    evaluation_table.loc[model_no] = [
        model_type,
        BATCH_SIZE,
        MAX_LEN,
        "AdamW",
        LR,
        EPS,
        n_val,
        "n/a",
        "n/a",
        val_loss,
        "n/a",
        "n/a",
        "n/a",
        val_f1,
        "n/a",
        dataset,
        train_data,
        device_name,
    ]
elif epoch == 4:
    evaluation_table.loc[model_no] = [
        model_type,
        BATCH_SIZE,
        MAX_LEN,
        "AdamW",
        LR,
        EPS,
        n_val,
        "n/a",
        "n/a",
        "n/a",
        val_loss,
        "n/a",
        "n/a",
        "n/a",
        val_f1,
        dataset,
        train_data,
        device_name,
    ]
else:
    print("Error: Check epoch")

evaluation_table.to_csv(
    "./results/sentiment_analysis/evaluation/evaluation_table.csv", index=False
)


# plot evaluation performance
plt.figure(figsize=(15, 8))
plt.title("Validation loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(val_loss_set)
# plt.show()
plt.savefig(
    "./results/sentiment_analysis/evaluation/{}/val_loss_{}_{}.png".format(
        dataset, model_no, epoch
    )
)

# plot validation losses last 20
l = n_val / BATCH_SIZE
l1 = int(l - 20)
l2 = int(l - 1)
plt.figure(figsize=(15, 8))
plt.title("Validation Loss - Last 20 Batches")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(val_loss_set[l1:l2])
# plt.show()
plt.savefig(
    "./results/sentiment_analysis/evaluation/{}/val_loss20_{}_{}.png".format(
        dataset, model_no, epoch
    )
)

# createFolder('data/evaluated/')
df.to_csv(
    "./data/processed/{}_wpredictions_{}_{}.csv".format(
        dataset, model_no, epoch
    )
)